In [9]:
from uatu.watchers import *
from sklearn.model_selection import train_test_split

In [10]:
X,Y = get_xy_from_dir('../data/Box000/', 0)

In [11]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.7)
train_dset = Dataset(X_train, Y_train, 20, shuffle = True, augment = True)
test_dset = Dataset(X_test, Y_test, 20, shuffle = True, augment = True)
data = (train_dset, test_dset, None)

In [12]:
device = 'GPU:0'

In [13]:
X.shape

(64, 64, 64, 64, 1)

In [14]:
def convnet_init_fn(inputs, training=False):
     #TODO add more customization
    initializer = tf.variance_scaling_initializer(scale=2.0)
    # TODO gotta be a better way to do this?
    #prob = tf.cond(training, lambda : 0.5, lambda : 1.0) #should i do some fancier tf stuff?
    axis = -1
    # NOTE ask waren if i need separate relus
    conv1_out = tf.layers.conv3d(inputs, 2, kernel_size=32, padding='same',
                                  kernel_initializer=initializer)
    bn1_out = tf.layers.batch_normalization(conv1_out, axis = axis, training=training)
    lr1_out = tf.nn.leaky_relu(bn1_out, alpha=0.01)
    ap1_out = tf.layers.average_pooling3d(lr1_out, pool_size=(31, 31, 31), strides = 2)
    conv2_out = tf.layers.conv3d(ap1_out, 12, kernel_size=16, padding='same',
                                  kernel_initializer=initializer)
    bn2_out = tf.layers.batch_normalization(conv2_out, axis = axis, training=training)
    lr2_out = tf.nn.leaky_relu(bn2_out, alpha=0.01)

    ap2_out = tf.layers.average_pooling3d(lr2_out, pool_size=(14, 14, 14), strides = 2)

    conv3_out = tf.layers.conv3d(ap2_out, 64, kernel_size= 4, padding='same',
                      kernel_initializer=initializer)
    bn3_out = tf.layers.batch_normalization(conv3_out, axis = axis, training=training)
    lr3_out = tf.nn.leaky_relu(bn3_out, alpha=0.01)
    #conv4_out = tf.layers.conv3d(lr3_out, 64, kernel_size=(4, 4, 4), padding='same')
    #                  kernel_initializer=initializer)
    #bn4_out = tf.layers.batch_normalization(conv4_out, axis = axis, training=training)
    #lr4_out = tf.nn.leaky_relu(bn4_out, alpha=0.01)
    #conv5_out =  tf.layers.conv3d(lr4_out, 128, kernel_size=(3, 3, 3), padding='same')
    #                  kernel_initializer=initializer)
    #bn5_out= tf.layers.batch_normalization(conv5_out, axis = axis, training=training)
    #lr5_out = tf.nn.leaky_relu(bn5_out, alpha=0.01)
    #conv6_out = tf.layers.conv3d(lr5_out, 128, kernel_size=(2, 2, 2), padding='same')
    #                  kernel_initializer=initializer)
    #bn6_out = tf.layers.batch_normalization(conv6_out, axis = axis, training= training)
    #lr6_out = tf.nn.leaky_relu(bn6_out, alpha=0.01)
    flat_out = tf.layers.flatten(lr3_out)
    dense1_out = tf.layers.dense(flat_out, 1024)# kernel_initializer=initializer)
    drop1_out = tf.layers.dropout(dense1_out, training=training)
    lr7_out =  tf.nn.leaky_relu(drop1_out, alpha=0.01)

    dense2_out = tf.layers.dense(lr7_out, 256)# kernel_initializer=initializer)
    drop2_out = tf.layers.dropout(dense2_out, training=training)
    lr8_out = tf.nn.leaky_relu(drop2_out, alpha=0.01)
    dense3_out = tf.layers.dense(lr8_out, 2)# kernel_initializer=initializer)

    return dense3_out

In [15]:
def optimizer_init_fn():
    return tf.train.AdamOptimizer(learning_rate=0.0005)

In [16]:
train(convnet_init_fn, optimizer_init_fn,data, device, fname = './shallow_test', num_epochs = 100)

Starting epoch 0
Iteration 0, loss = 27.0135
[[-2.8829753 -3.3125196]
 [-2.8559747 -3.3128068]
 [-2.8205032 -3.2685604]
 [-2.85831   -3.3251374]
 [-2.8978405 -3.384361 ]
 [-2.8732305 -3.3076766]
 [-2.8873951 -3.3389363]
 [-2.881757  -3.3318517]
 [-2.8541846 -3.3105197]
 [-2.8668365 -3.2957418]
 [-2.861541  -3.2991517]
 [-2.8773334 -3.3150394]
 [-2.8296435 -3.2578583]
 [-2.824477  -3.2794225]
 [-2.843998  -3.2910695]
 [-2.8444164 -3.2973566]
 [-2.840159  -3.2916236]
 [-2.8755472 -3.3289256]
 [-2.8305633 -3.2481384]
 [-2.8978004 -3.342859 ]] 
[[0.343403 0.070558]
 [0.343403 0.070558]
 [0.343403 0.070558]
 [0.343403 0.070558]
 [0.343403 0.070558]
 [0.343403 0.070558]
 [0.343403 0.070558]
 [0.343403 0.070558]
 [0.343403 0.070558]
 [0.343403 0.070558]
 [0.343403 0.070558]
 [0.343403 0.070558]
 [0.343403 0.070558]
 [0.343403 0.070558]
 [0.343403 0.070558]
 [0.343403 0.070558]
 [0.343403 0.070558]
 [0.343403 0.070558]
 [0.343403 0.070558]
 [0.343403 0.070558]]
******************************
[

[[0.6134796  0.27993885]
 [0.605066   0.2798324 ]
 [0.6194141  0.2821774 ]
 [0.6284263  0.28924593]
 [0.62459695 0.28596088]] 
[[0.343403 0.070558]
 [0.343403 0.070558]
 [0.343403 0.070558]
 [0.343403 0.070558]
 [0.343403 0.070558]]
******************************
()
Starting epoch 31
Starting epoch 32
Starting epoch 33
Starting epoch 34
Starting epoch 35
Starting epoch 36
Starting epoch 37
Starting epoch 38
Starting epoch 39
Starting epoch 40
Iteration 40, loss = 5.6786
[[0.07615028 0.35152176]
 [0.075823   0.34599805]
 [0.07961587 0.3428298 ]
 [0.07850967 0.34732398]
 [0.07867166 0.35310036]
 [0.07620485 0.34860453]
 [0.07997278 0.3438675 ]
 [0.07643928 0.3425801 ]
 [0.07669198 0.33898443]
 [0.08004154 0.34649214]
 [0.07500457 0.3503811 ]
 [0.07844097 0.35215756]
 [0.07659318 0.34890026]
 [0.07664075 0.34122983]
 [0.07632221 0.3440852 ]
 [0.0748244  0.3420471 ]
 [0.07589722 0.34097096]
 [0.07698567 0.3479551 ]
 [0.07728463 0.3383326 ]
 [0.07864913 0.34937623]] 
[[0.343403 0.070558]
 [

[[0.6390594  0.09002645]
 [0.63631326 0.08743566]
 [0.6256145  0.08703846]
 [0.63523513 0.08758135]
 [0.6359642  0.0879555 ]
 [0.62034464 0.08652993]
 [0.6371857  0.08993466]
 [0.6365064  0.08908778]
 [0.6395537  0.08871157]
 [0.6458502  0.08981018]
 [0.6229546  0.08744795]
 [0.6331498  0.08833276]
 [0.6487312  0.09049238]
 [0.64170843 0.08873416]
 [0.6166082  0.08581077]
 [0.644482   0.09157312]
 [0.63181716 0.08760118]
 [0.6348459  0.08847502]
 [0.6332102  0.08837519]
 [0.6444865  0.08986741]] 
[[0.343403 0.070558]
 [0.343403 0.070558]
 [0.343403 0.070558]
 [0.343403 0.070558]
 [0.343403 0.070558]
 [0.343403 0.070558]
 [0.343403 0.070558]
 [0.343403 0.070558]
 [0.343403 0.070558]
 [0.343403 0.070558]
 [0.343403 0.070558]
 [0.343403 0.070558]
 [0.343403 0.070558]
 [0.343403 0.070558]
 [0.343403 0.070558]
 [0.343403 0.070558]
 [0.343403 0.070558]
 [0.343403 0.070558]
 [0.343403 0.070558]
 [0.343403 0.070558]]
******************************
[[0.6234083  0.08646756]
 [0.6138407  0.085879